# Regras

Para essa base de dados disponibilizada (`IDH_Nacoes_2020.xlsx`), resolva os problemas abaixo.
Lembre-se de que suas resoluções **SEMPRE** deve conter os quatro elementos seguintes:
  * Justificativa das técnicas usadas paa resolução
  * Implementação da solução
  * Resposta - Opinião própria
  * Comentário/justificativa do resultado

# Análise sobre os indicadores de desenvolvimento humano

O arquivo anexado `IDH_Nacoes_2020.xlsx` trás um recorte do indicadores de desenvolvimento humano computados de 189 nações do nosso globo.

Os indicadores observados foram:
  * Expectativa de vida ao nascer
  * Expectativa de anos de escolaridade
  * Média de anos de escolaridade
  * Renda nacional bruta (RNB) per capita
  
E, com base neles, foi calculado o Índice de Desenvolvimento Humano (HDI).

## Problema 1

**Deseja-se criar um modelo preditivo do IDH dado os quatro valores de: Expectativa de vida ao nascer, Expectativa de anos de escolaridade, Média de anos de escolaridade e Renda nacional bruta (RNB) per capita.**

**Utilizando Aprendizagem de Máquina, treine um modelo que resolva o problema proposto e prediga os valores de IDH para os padrões de teste abaixo:**

| Padrão de teste | Expectativa de vida ao nascer | Expectativa de anos de escolaridade | Média de anos de escolaridade | Renda nacional bruta (RNB) per capita |
| --- | --- | --- | --- | --- |
| 1 | 72,1	|	12,1	|	7,3	|	14.869 |
| 2 | 75,4	|	13,6	|	8,1	|	14.710 |
| 3 | 74,4	|	14,7	|	10,4	|	17.939 |
| 4 | 75,6	|	14,6	|	8,7	|	14.812 |
| 5 | 69,9	|	11,7	|	6,5	|	6.532 |
| 6 | 61,5	|	10,1	|	5,8	|	3.686 |

### Justificativa das técnicas usadas paa resolução

In [ ]:
Foi utlizada a métrica 'R-squared (R²) que representa a proporção da variancia peara uma variável dependente, que é explicada por uma variável ou variáveis 
independentes em um modelo de regressão.
No caso, se o R² de um modelo é 0.50, então aproximadamente metade dos resultados estarão corretos.

Todas as respostas utilizadas para o exercício serão feitas utilizando a linguagem 'C#' e ferramentas do .NET da Microsoft voltadas para Aprendizagem de Máquina.

### Implementação da solução

In [ ]:
// Arquivo principal para introdução dos dados e recebimento do resultado

using MLFaculdade;

//Load sample data
var sampleData = new PredictIDH.ModelInput()
{
    Países = @"Norway",
    Expectativa_de_vida_ao_nascer = 82.4F,
    Expectativa_de_anos_de_escolaridade = 18.1F,
    Média_de_anos_de_escolaridade = 12.9F,
    Renda_nacional_bruta__RNB__per_capita = 66494F,
};

//Load model and predict output
var result = PredictIDH.Predict(sampleData);

In [ ]:
// Arquivo que contém o método publico que irá criar e retornar a previsão

using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Linq;
using System.IO;
using System.Collections.Generic;
namespace MLFaculdade
{
    public partial class PredictIDH
    {
        /// <summary>
        /// model input class for PredictIDH.
        /// </summary>
        #region model input class
        public class ModelInput
        {
            [ColumnName(@"Países")]
            public string Países { get; set; }

            [ColumnName(@"Indice Desenvolvimento Humano (IDH)")]
            public float Indice_Desenvolvimento_Humano__IDH_ { get; set; }

            [ColumnName(@"Expectativa de vida ao nascer")]
            public float Expectativa_de_vida_ao_nascer { get; set; }

            [ColumnName(@"Expectativa de anos de escolaridade")]
            public float Expectativa_de_anos_de_escolaridade { get; set; }

            [ColumnName(@"Média de anos de escolaridade")]
            public float Média_de_anos_de_escolaridade { get; set; }

            [ColumnName(@"Renda nacional bruta (RNB) per capita")]
            public float Renda_nacional_bruta__RNB__per_capita { get; set; }

        }

        #endregion

        /// <summary>
        /// model output class for PredictIDH.
        /// </summary>
        #region model output class
        public class ModelOutput
        {
            public float Score { get; set; }
        }
        #endregion

        private static string MLNetModelPath = Path.GetFullPath("PredictIDH.zip");

        public static readonly Lazy<PredictionEngine<ModelInput, ModelOutput>> PredictEngine = new Lazy<PredictionEngine<ModelInput, ModelOutput>>(() => CreatePredictEngine(), true);

        /// <summary>
        /// Use this method to predict on <see cref="ModelInput"/>.
        /// </summary>
        /// <param name="input">model input.</param>
        /// <returns><seealso cref=" ModelOutput"/></returns>
        public static ModelOutput Predict(ModelInput input)
        {
            var predEngine = PredictEngine.Value;
            return predEngine.Predict(input);
        }

        private static PredictionEngine<ModelInput, ModelOutput> CreatePredictEngine()
        {
            var mlContext = new MLContext();
            ITransformer mlModel = mlContext.Model.Load(MLNetModelPath, out var _);
            return mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(mlModel);
        }
    }
}


In [ ]:
// Arquivo que contém a configuração de treinamento do sistema.

using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML;

namespace MLFaculdade
{
    public partial class PredictIDH
    {
        public static ITransformer RetrainPipeline(MLContext context, IDataView trainData)
        {
            var pipeline = BuildPipeline(context);
            var model = pipeline.Fit(trainData);

            return model;
        }

        /// <summary>
        /// build the pipeline that is used. Use this function to retrain model.
        /// </summary>
        /// <param name="mlContext"></param>
        /// <returns></returns>
        public static IEstimator<ITransformer> BuildPipeline(MLContext mlContext)
        {
            // Data process configuration with pipeline data transformations
            var pipeline = mlContext.Transforms.ReplaceMissingValues(new []{new InputOutputColumnPair(@"Expectativa de vida ao nascer", @"Expectativa de vida ao nascer"),new InputOutputColumnPair(@"Expectativa de anos de escolaridade", @"Expectativa de anos de escolaridade"),new InputOutputColumnPair(@"Média de anos de escolaridade", @"Média de anos de escolaridade"),new InputOutputColumnPair(@"Renda nacional bruta (RNB) per capita", @"Renda nacional bruta (RNB) per capita")})      
                                    .Append(mlContext.Transforms.Text.FeaturizeText(@"Países", @"Países"))      
                                    .Append(mlContext.Transforms.Concatenate(@"Features", new []{@"Expectativa de vida ao nascer",@"Expectativa de anos de escolaridade",@"Média de anos de escolaridade",@"Renda nacional bruta (RNB) per capita",@"Países"}))      
                                    .Append(mlContext.Transforms.NormalizeMinMax(@"Features", @"Features"))      
                                    .Append(mlContext.Regression.Trainers.LbfgsPoissonRegression(l1Regularization:0.342754410622423F,l2Regularization:1.51801721911537F,labelColumnName:@"Indice Desenvolvimento Humano (IDH)",featureColumnName:@"Features"));

            return pipeline;
        }
    }
}


### Resposta - Opinião própria

In [ ]:
Tal cenário deu como resultado 0.89 de acurácia quando utilizado na técnica R².
Quando colocado para executar, as respostas dadas pelo sistema foram próximas, mas sempre eram arrendondadas para cima quando comparados à tabela, como, por exemplo, quando
foi dado para executar os dados da Noruega, o IDH de resultado foi 0.960, quando na tabela está 0.957.

### Comentário/justificativa do resultado

In [ ]:
Os resultados mostrados pelo cenário indicam que com uma acurácia tão alta, a maioria dos resultados apresentados seguirão os resultados dados na tabela, com um índice de erro de
alguns pontos para cima, visando arredondar o número para a casa mais perto do zero.

## Problema 2

**Através dos dados, um pesquisador deseja entender como os indicadores contribuem para a formação do IDH.**

**Com uso de AM, apresente uma alternativa para análise desses indicadores e indique qual é a ordenação descendente deles em grau de importância quanto a formação do valor de IDH**

### Justificativa das técnicas usadas paa resolução

### Implementação da solução

### Resposta - Opinião própria

### Comentário/justificativa do resultado

## Problema 3

**Deseja-se classificar os países em quatro categorias, de acordo com seu valor de IDH. As categorias são: IDH muito alto, IDH alto, IDH médio e IDH baixo.**

**Analise os dados e apresente uma solução coerente que use AM para determinar a faixa de valores de IDH de cada categoria.**

**Além disso, utilizando suas soluções, os padrões de teste do `Problema 1` recebem qual categoria de IDH?**

### Justificativa das técnicas usadas paa resolução

### Implementação da solução

### Resposta - Opinião própria

### Comentário/justificativa do resultado